In [47]:
from datasets import load_dataset
sentiment_dataset = load_dataset("hezarai/sentiment-dksf")

In [48]:
sentiment_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 28602
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2315
    })
})

In [49]:
train_df = sentiment_dataset['train'].to_pandas()
test_df = sentiment_dataset['test'].to_pandas()

In [4]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28602 entries, 0 to 28601
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    28602 non-null  object
 1   label   28602 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 447.0+ KB


In [6]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2315 entries, 0 to 2314
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    2315 non-null   object
 1   label   2315 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 36.3+ KB


In [7]:
train_df.to_csv('dataset\\train.csv')
test_df.to_csv('dataset\\test.csv')

In [5]:
train_df.head()

,text,label
0,شیشه هاش همون روز اول شکست,0
1,تمیز -مرتب- چقدر پیک آن تایم و مؤدب …سپاس بسیا...,1
2,نظر من هیچ گاه این محصول رو خریداری نکنید اگه ...,0
3,برنج ایرانی سفارش دادم افتضاح بود با هندی هیچ ...,0
4,سلام اصلا راضی نیستم یعنی ارزش خرید نداره,0


In [6]:
import hazm
import re

In [7]:
punctuation = "?.؟!،,"
normalizer = hazm.Normalizer()
def preprocessing(item):
    output = normalizer.normalize(item)
    output = output.replace("\_", "\u200c")
#     output = output.replace("\u200c", "")
    output = re.sub(r"LINK([^ ])*|TAG|ID|\-|@|LINK|[A-Za-z]", "", output)
    for i in punctuation:
        p = "\\" + i + "{2,}"
        output = re.sub(p, i, output)
#     output = re.sub("\?|\.|\؟|\!|\،|\,", "", output)
    # output = removeEmojies(output)
    # emoj = re.compile(e)
    # for i in emoj.findall(output):
    #     allemojies.add(i)
    output = re.sub("\s+", " ", output).strip()
    return output

In [50]:
train_df.text = train_df.text.apply(preprocessing)
test_df.text = test_df.text.apply(preprocessing)

In [9]:
train_df.text.head()

0                           شیشه هاش همون روز اول شکست
1    تمیز مرتب چقدر پیک آن تایم و مؤدب …سپاس بسیار ...
2    نظر من هیچ‌گاه این محصول رو خریداری نکنید اگه ...
3    برنج ایرانی سفارش دادم افتضاح بود با هندی هیچ ...
4            سلام اصلا راضی نیستم یعنی ارزش خرید نداره
Name: text, dtype: object

In [10]:
test_df.text.head()

0    عطره و بیشتر به درد لباس میخوره، چون اصلا پخش ...
1    اصلا بدرد نمیخوره. بعد از دو سه روز کنده میشه....
2    غذا بسیار با کیفیت بود اما متاسفانه دوتا سفارش...
3    غذا خیلی خوب بود، خیلی سریع رسید و قیمتشم خیلی...
4            قیمت رو بالا ببرید کیفیت رو کم نکنید لطفا
Name: text, dtype: object

In [51]:
from sklearn.model_selection import train_test_split

In [52]:
val_df, test_df = train_test_split(test_df, test_size=0.4)

In [53]:
train_df.shape

(28602, 2)

In [54]:
val_df.shape

(1389, 2)

In [55]:
test_df.shape

(926, 2)

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer
# tokenizer = hazm.word_tokenize()
vectorizer = TfidfVectorizer()

In [15]:
sentences = [sentenc for sentenc in train_df.text]

In [16]:
sentences

['شیشه هاش همون روز اول شکست',
 'تمیز مرتب چقدر پیک آن تایم و مؤدب …سپاس بسیار عالی فقط کاشکی رنگ دستمال همونی می\u200cبود که در شکل انتخاب کردیم … باز هم سپاس بسیار عالی از سرعت و کیفیت',
 'نظر من هیچ\u200cگاه این محصول رو خریداری نکنید اگه می\u200cخواهید شانستون رو امتحان کنین ببینین کارت هاتون ۱۰۸ تا یا ۵۲ هستش راه باز هست بعضی\u200cها میگن ۱۰۸ کارت داره بعضی\u200cها هم میگن ۵۴ تا کارت داره در اصل این محصول باید ۱۰۸ کارت باشه که برای من ۵۲ تا دراومد و فارسی هم هستش و برای یک کالای دیگه هستش که قیمتش ارزونتره',
 'برنج ایرانی سفارش دادم افتضاح بود با هندی هیچ فرقی نمی\u200cکرد',
 'سلام اصلا راضی نیستم یعنی ارزش خرید نداره',
 'من اینو برای هدیه دادن گرفتم همون طور که انتظار داشتم ریز و قشنگه و از هرکی پرسیدم خوشش اومد، از دوام و ماندگاریش نمی\u200cدونم چون تازه خریدم.',
 'حتما پیشنهاد می\u200cکنم بخرید.',
 'من این محصولو پارسال خریدم اصلا خوب نیس ویبره آن ماساژ که نمیده هیچ سردرد میاره. خیلی از خریدش پشیمونم. با تشکر از دیجی',
 'طعم بال عالیه پیتزا هم طعمش خوبه اما هر بار پیتزا سرد میر

In [17]:
hazm_tokenizer = hazm.WordTokenizer(replace_numbers=True, replace_ids=True)

In [39]:
train_token_sentiment = [hazm_tokenizer.tokenize(sentence) for sentence in sentences]

In [40]:
train_token_sentiment[-1]

['ایرانسل',
 'طرح\u200cهای',
 'جدید',
 'از',
 'کی',
 'میاد',
 '،',
 'میخوام',
 'زمستون',
 'گرمی',
 'برای',
 'ما',
 'کاربرها',
 'رقم',
 'بزنی',
 '�']

In [23]:
from gensim.models.word2vec import Word2Vec

In [25]:
import numpy as np

In [41]:
model = Word2Vec(sentences=train_token_sentiment, window=10, min_count=5, seed=42, workers=5)

In [29]:
def sentence_to_vector(sentence, model):
    words = [word for word in sentence if word in model.wv.key_to_index]
    if len(words) == 0:
        return np.zeros(model.vector_size)
    return np.mean([model.wv[word] for word  in words], axis=0)

In [44]:
train_sentiment_vectors = [sentence_to_vector(token, model) for token in train_token_sentiment]

In [45]:
train_sentiment_vectors[-1]

array([-0.04283768,  0.07398508, -0.01486063,  0.13644958, -0.4004199 ,
        0.17001086,  0.10924066,  0.22881576, -0.05765148, -0.11775303,
       -0.32072628, -0.05574371,  0.23663588,  0.45176047, -0.2465429 ,
       -0.26618788,  0.12822923, -0.43392107, -0.19966446,  0.13838829,
        0.17935942,  0.11853928, -0.10613549,  0.47442043,  0.12588902,
       -0.01788238, -0.17507496, -0.18873452, -0.07845535,  0.26581675,
        0.05538391, -0.4173277 ,  0.23017852,  0.0785614 ,  0.45518157,
       -0.15120856, -0.08315716, -0.20551091, -0.15494736,  0.5046135 ,
        0.27092278, -0.3900772 ,  0.2832494 , -0.23420982,  0.40613106,
       -0.12681207,  0.35254186,  0.42882985,  0.01522604, -0.5047128 ,
       -0.28832567, -0.731398  ,  0.10944445,  0.02264187,  0.47063872,
       -0.03453654, -0.19790599,  0.27057177, -0.10581864,  0.6966073 ,
       -0.17045678, -0.10976551, -0.30233067,  0.19839187, -0.03282093,
       -0.20361668,  0.08752434, -0.42000052, -0.00470053,  0.03

In [56]:
def convert_text_to_vector(text_df, model):
    sentences = [sentenc for sentenc in text_df.text]
    token = [hazm_tokenizer.tokenize(sentence) for sentence in sentences]
    vectors = [sentence_to_vector(token, model) for token in token]
    return vectors

train_vectors = convert_text_to_vector(train_df, model)
val_vectors = convert_text_to_vector(val_df, model)
test_vectors = convert_text_to_vector(test_df, model)    

In [57]:
print(np.shape(train_vectors))
print(np.shape(val_vectors))
print(np.shape(test_vectors))

(28602, 100)
(1389, 100)
(926, 100)


In [35]:
tokens = []
for sentenc in sentences:
    token = hazm.word_tokenize(sentenc)
    tokens.append(token)
tokens =  [i for j in tokens for i in j]

In [36]:
tokens

['شیشه',
 'هاش',
 'همون',
 'روز',
 'اول',
 'شکست',
 'تمیز',
 'مرتب',
 'چقدر',
 'پیک',
 'آن',
 'تایم',
 'و',
 'مؤدب',
 '…سپاس',
 'بسیار',
 'عالی',
 'فقط',
 'کاشکی',
 'رنگ',
 'دستمال',
 'همونی',
 'می\u200cبود',
 'که',
 'در',
 'شکل',
 'انتخاب',
 'کردیم',
 '…',
 'باز',
 'هم',
 'سپاس',
 'بسیار',
 'عالی',
 'از',
 'سرعت',
 'و',
 'کیفیت',
 'نظر',
 'من',
 'هیچ\u200cگاه',
 'این',
 'محصول',
 'رو',
 'خریداری',
 'نکنید',
 'اگه',
 'می\u200cخواهید',
 'شانستون',
 'رو',
 'امتحان',
 'کنین',
 'ببینین',
 'کارت',
 'هاتون',
 '۱۰۸',
 'تا',
 'یا',
 '۵۲',
 'هستش',
 'راه',
 'باز',
 'هست',
 'بعضی\u200cها',
 'میگن',
 '۱۰۸',
 'کارت',
 'داره',
 'بعضی\u200cها',
 'هم',
 'میگن',
 '۵۴',
 'تا',
 'کارت',
 'داره',
 'در',
 'اصل',
 'این',
 'محصول',
 'باید',
 '۱۰۸',
 'کارت',
 'باشه',
 'که',
 'برای',
 'من',
 '۵۲',
 'تا',
 'دراومد',
 'و',
 'فارسی',
 'هم',
 'هستش',
 'و',
 'برای',
 'یک',
 'کالای',
 'دیگه',
 'هستش',
 'که',
 'قیمتش',
 'ارزونتره',
 'برنج',
 'ایرانی',
 'سفارش',
 'دادم',
 'افتضاح',
 'بود',
 'با',
 'هندی',
 'هیچ',
 'ف

In [37]:
vectorizer.fit(tokens)

TfidfVectorizer()

In [42]:
x = vectorizer.transform(hazm.word_tokenize(train_df['text'][0]))

In [44]:
x.shape

(6, 36193)

In [46]:
print(x.toarray())

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [47]:
from sklearn.feature_extraction.text import CountVectorizer

In [50]:
count_vectorizer = CountVectorizer()
count_vectorizer.fit(tokens)
y = count_vectorizer.transform(hazm.word_tokenize(train_df['text'][0]))
print(y.toarray())

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [51]:
y.shape

(6, 36193)